In [1]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

Using TensorFlow backend.


In [2]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')

# Sentiment data

In [3]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [4]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [5]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [6]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [7]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')
    length.append(len(data['CONTENT'][i]))
    if data['SENTIMENT'][i] == 'Irrelevant':
        data['SENTIMENT'][i] = 'Neutral'
length = np.array(length)

In [8]:
mypath = './data/processed/'
if not os.path.exists(mypath):
    os.makedirs(mypath)
for the_file in os.listdir(mypath):
    file_path = os.path.join(mypath, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [63]:
for x in data['SENTIMENT'].unique():
    np.savetxt(mypath+ x.replace('/',' ') +'.utf8',\
                   data[data['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [65]:
data['SENTIMENT'].value_counts()

Neutral     14871
Positive      465
Negative      287
Name: SENTIMENT, dtype: int64

In [66]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model.512')

In [67]:
len(w2vModel.wv.vocab)

3570

In [68]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [53]:
vocab_obj = w2vModel.wv.vocab
word = u'河北'
print word
print vocab_obj[word].index

河北
3038


In [36]:
dianping_json = pd.read_table('./data/reviews.txt')

In [55]:
x = pd.read_json(dianping_json.loc[0][0].split('^')[1])

In [ ]:
x = []
for i in range(len(dianping_json)):
    x.append(dianping_json.loc[i][0].split('^')[1])